# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../WeatherPy/Weather_Data.csv"

weather_data_df = pd.read_csv(file_path)

weather_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Clyde River,70.47,-68.59,42.84,76,0,5.32,CA
1,Salamanca,40.83,-6.00,72.00,68,82,6.93,ES
2,Antofagasta,-23.65,-70.40,55.40,76,90,2.24,CL
3,Mar del Plata,-38.00,-57.56,42.80,56,52,11.41,AR
4,Srednekolymsk,67.45,153.68,71.53,41,0,7.25,RU
...,...,...,...,...,...,...,...,...
551,Alofi,-19.06,-169.92,73.40,94,57,16.11,NU
552,Port-Gentil,-0.72,8.78,77.00,88,75,12.48,GA
553,Bonthe,7.53,-12.51,75.13,91,100,11.43,SL
554,Pop,40.87,71.11,91.40,24,20,6.93,UZ


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_data_df[["Lat", "Lng"]].astype(float)
humidity = weather_data_df["Humidity"]

In [6]:
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 2)

fig.add_layer(heat_layer)

#markers = gmaps.marker_layer(locations)

#fig.add_layer(markers)


fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Conditions:
#Temp above 85 Degrees F
#Cloudiness is zero
#Humidity less than 50
weather_data_df_filtered = weather_data_df.loc[weather_data_df['Max Temp'] > 85]
weather_data_df_filtered = weather_data_df_filtered.loc[weather_data_df_filtered['Cloudiness'] == 0]
weather_data_df_filtered = weather_data_df_filtered.loc[weather_data_df_filtered['Humidity'] < 30]
weather_data_df_filtered = weather_data_df_filtered.reset_index()
weather_data_df_filtered
#newdf = df.loc[(df.origin == "JFK") & (df.carrier == "B6")]
#weather_data_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,12,Chaman,30.92,66.45,98.42,9,0,17.72,PK
1,32,Yangi Marg`ilon,40.43,71.72,89.60,25,0,4.70,UZ
2,173,Arys,42.43,68.80,96.80,18,0,15.66,KZ
3,186,Sohag,26.56,31.69,96.80,22,0,19.46,EG
4,220,Manbij,36.53,37.95,95.00,24,0,14.05,SY
5,241,Tiznit Province,29.58,-9.50,93.31,23,0,5.59,MA
6,269,Ust-Kuyga,70.02,135.60,85.06,21,0,8.34,RU
7,285,Aswān,24.09,32.91,102.38,15,0,16.51,EG
8,359,Dehlorān,32.69,47.27,110.70,6,0,20.94,IR
9,389,Abu Kamal,34.45,40.92,102.54,17,0,12.37,SY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather_data_df_filtered
hotel_df["Hotel Name"] = ""
hotel_df


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,12,Chaman,30.92,66.45,98.42,9,0,17.72,PK,
1,32,Yangi Marg`ilon,40.43,71.72,89.60,25,0,4.70,UZ,
2,173,Arys,42.43,68.80,96.80,18,0,15.66,KZ,
3,186,Sohag,26.56,31.69,96.80,22,0,19.46,EG,
4,220,Manbij,36.53,37.95,95.00,24,0,14.05,SY,
5,241,Tiznit Province,29.58,-9.50,93.31,23,0,5.59,MA,
6,269,Ust-Kuyga,70.02,135.60,85.06,21,0,8.34,RU,
7,285,Aswān,24.09,32.91,102.38,15,0,16.51,EG,
8,359,Dehlorān,32.69,47.27,110.70,6,0,20.94,IR,
9,389,Abu Kamal,34.45,40.92,102.54,17,0,12.37,SY,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    City = row["City"]
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    hotel_info = requests.get(base_url, params=params)
    
    hotel_info = hotel_info.json()
    
    print(f"Searching for Hotel in {City}")
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Sorry, No Nearby Hotels Found..")    
        
      
    
    print("-----------------------------")
    
    
    


Searching for Hotel in Chaman
-----------------------------
Searching for Hotel in Yangi Marg`ilon
-----------------------------
Searching for Hotel in Arys
-----------------------------
Searching for Hotel in Sohag
-----------------------------
Searching for Hotel in Manbij
-----------------------------
Searching for Hotel in Tiznit Province
Sorry, No Nearby Hotels Found..
-----------------------------
Searching for Hotel in Ust-Kuyga
Sorry, No Nearby Hotels Found..
-----------------------------
Searching for Hotel in Aswān
-----------------------------
Searching for Hotel in Dehlorān
-----------------------------
Searching for Hotel in Abu Kamal
-----------------------------
Searching for Hotel in Karatau
Sorry, No Nearby Hotels Found..
-----------------------------
Searching for Hotel in Arlit
-----------------------------
Searching for Hotel in Korla
-----------------------------
Searching for Hotel in Farah
Sorry, No Nearby Hotels Found..
-----------------------------


In [22]:
hotel_df
          
    

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,12,Chaman,30.92,66.45,98.42,9,0,17.72,PK,Sakhy Hotal Wah Mosafir Khana
1,32,Yangi Marg`ilon,40.43,71.72,89.60,25,0,4.70,UZ,Hotel Mehmon Saroy
2,173,Arys,42.43,68.80,96.80,18,0,15.66,KZ,Гостиница МАДАНИ
3,186,Sohag,26.56,31.69,96.80,22,0,19.46,EG,فندق أغابى
4,220,Manbij,36.53,37.95,95.00,24,0,14.05,SY,Manbej City Hotel
5,241,Tiznit Province,29.58,-9.50,93.31,23,0,5.59,MA,
6,269,Ust-Kuyga,70.02,135.60,85.06,21,0,8.34,RU,
7,285,Aswān,24.09,32.91,102.38,15,0,16.51,EG,Sofitel Legend Old Cataract Aswan
8,359,Dehlorān,32.69,47.27,110.70,6,0,20.94,IR,هتل سعید
9,389,Abu Kamal,34.45,40.92,102.54,17,0,12.37,SY,محطة التلفاز


In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
